In [ ]:
from datasets import SOCOFingGender, SOCOFingFingers, SOCOFingSubjects, SOCOFingHeands, QFing
from FPMLmodule.backbones import ResNet50
from FPMLmodule.classifiers import ResNetClassifier
from FPMLmodule.fpml import FPML 
import FPMLmodule.utils as utils
import numpy as np
from matplotlib import pyplot as plt
# from keras import layers, Sequential, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

In [ ]:
# Global Config
seed=9
img_dim = (120, 120, 3)
img_height, img_width, img_channels = img_dim
batch_size = 32

# Dataset configuration
dsConfig = {
    'batchSize': batch_size, 
    'parallelTune': tf.data.AUTOTUNE, 
    'split': [0.7, 0.15, 0.15], 
    'inputDim': img_dim, 
    'seed': seed, 
    'shuffle': True
    }
weights = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Model Config
learning_rate = 0.001
epochs_find_best = 10
epochs_best = 100



In [ ]:
SOCOGender = SOCOFingGender(**dict(dsConfig, sampling=SOCOFingGender.UNDER_SAMPLING))
SOCOSubjects = SOCOFingSubjects(**dsConfig)
SOCOFingers = SOCOFingFingers(**dsConfig)
QualityFing = QFing(**dsConfig)
SOCOHeands = SOCOFingHeands(**dsConfig)

genderDS = SOCOGender.create()
subjectDS = SOCOSubjects.create()
fingersDS = SOCOFingers.create()
heandsDS = SOCOHeands.create()
qfingDS = QualityFing.create()

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import preprocessing


trainAug = models.Sequential([
	preprocessing.Rescaling(scale=1.0 / 255),
	preprocessing.RandomFlip("horizontal_and_vertical"),
	preprocessing.RandomZoom(
		height_factor=(-0.05, -0.15),
		width_factor=(-0.05, -0.15)),
	preprocessing.RandomRotation(0.3)
])


# augmanted = data_augmentation(genderDS[0])


In [ ]:
utils.displayDsSamples(heandsDS[0], 'Heands - Train', (5, 5), classNames=SOCOHeands.classNames)

In [ ]:
train_ds, test_ds, val_ds = genderDS
rn50 = ResNet50(img_dim, weights=weights, trainable=False)
rnc = ResNetClassifier(2, "softmax")
fpml = FPML(rn50, rnc, "", img_dim)
model = fpml.create(Adam, learning_rate, 'binary_crossentropy', 'accuracy')
model_history = model.fit(train_ds, validation_data=val_ds, epochs=10)
test_history = model.evaluate(test_ds)

In [ ]:
displayTrainProgress(model_history)
test_acc = displayConfusion(test_ds, model)